In [1]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [2]:
bd.projects.set_current('ei38-teaching-25')
ipcc=('IPCC 2013', 'climate change', 'GWP 100a')

In [3]:
ei = bd.Database('ei 3.8 cutoff')
activity= ei.get('6a7848cd768f5163834eec59c80de49b')
ei_pack=ei.datapackage()

In [4]:
lca1 = bc.LCA({activity: 1}, ipcc)
lca1.lci()
lca1.lcia()
lca1.score

2.477976880419227

In [5]:
#[act for act in ei if 'market for acrylic acid' in  act['name']][1].id

In [6]:
heat_acts=[act for act in ei if 'heat production' in  act['name']
#and ('RER' in act['location'] or 'DE' in act['location'] or 'Europe without Switzerland' in act['location'])
#and 'heat' in act['reference product'] 
and 'heat pump' not in act['name']
and 'biomethane' not in act['name']
#and ('natural gas' in act['name'] or 'anthracite' in act['name'])
]

In [7]:
new_heat_supply=[act for act in ei if 'market for electricity, medium voltage' in act['name'] and 'DE' in act['location']][0]
new_heat_supply

'electricity, from municipal waste incineration to generic market for electricity, medium voltage' (kilowatt hour, DE, None)

In [8]:
exchange =list(activity.technosphere())[0]
a= np.array((exchange.input.id, exchange.output.id), bwp.INDICES_DTYPE)

In [9]:
np.where(ei_pack.data[5] == a)

(array([157541], dtype=int64),)

In [10]:
index=[]
flow_list=[]
global depth
depth=0
max_depth=2

In [11]:
ei_pack.data[5][3]['row']

4428

In [12]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [13]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [14]:
def recurse_supply(act):
    global depth
    for exc in act.technosphere():
        if exc.input in heat_acts:
            #flow_list.append(np.array((exc.input.id, exc.output.id), bwp.INDICES_DTYPE))
            flow_list.append((exc.input.id, exc.output.id))
            #print(exc.input)
        elif depth< max_depth:
            depth +=1 
            recurse_supply(exc.input)
    depth -= 1
        

In [15]:
flow_list=[]
global depth
depth=0
max_depth=3
recurse_supply(activity)

In [16]:
new_flow_list=[]
for exc in flow_list:
    new_flow_list.append((new_heat_supply.id, exc[1]))
    #new_flow_list.append(np.array((new_heat_supply.id, exc['col']), bwp.INDICES_DTYPE))


In [17]:
empty_data= np.zeros(len(flow_list))

In [32]:
len(ei_pack.resources)

9

In [18]:
#new_flow_list.append(np.array((new_heat_supply.id, exc['col']), bwp.INDICES_DTYPE))

In [19]:
data_list=[]
for exc in flow_list:  
    data_list.append(ei_pack.data[6][np.where(ei_pack.data[5] == np.array((exc), bwp.INDICES_DTYPE))][0])

In [43]:
flip=np.zeros(len(data_list)*2, dtype=bool)

In [44]:
indices = np.array(flow_list + new_flow_list, bwp.INDICES_DTYPE)
data = np.array(list(empty_data) + data_list)

In [45]:
dp = bwp.create_datapackage()

In [46]:

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip=flip,
    name="new_heat_supply",
)

In [47]:
_, data_objs, _ = bd.prepare_lca_inputs({activity: 1}, ipcc)

In [55]:
data_objs[2].metadata

{'profile': 'data-package',
 'name': 'ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf.zip',
 'id': '2839247ce3404fa29f39c82ec8276f8d',
 'licenses': [{'name': 'ODC-PDDL-1.0',
   'path': 'http://opendatacommons.org/licenses/pddl/',
   'title': 'Open Data Commons Public Domain Dedication and License v1.0'}],
 'resources': [{'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stream',
   'name': 'IPCC_2013_climate_change_GWP_100a_matrix_data.indices',
   'matrix': 'characterization_matrix',
   'kind': 'indices',
   'path': 'IPCC_2013_climate_change_GWP_100a_matrix_data.indices.npy',
   'group': 'IPCC_2013_climate_change_GWP_100a_matrix_data',
   'nrows': 211,
   'global_index': 1,
   'category': 'vector'},
  {'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stream',
   'name': 'IPCC_2013_climate_change_GWP_100a_matrix_data.data',
   'matrix': 'characterization_matrix',
   'kind': 'data',
   'path': 'IPCC_2013_climate_change_GWP_

In [56]:
lca2 = bc.LCA({activity.id: 1}, data_objs=data_objs)
lca2.lci()
lca2.lcia()

In [57]:
print(lca1.score, lca2.score)

2.477976880419227 2.477976880419227


In [60]:
dp.data[1]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      